# Function Annotations

In this code snippet, we demonstrate function decorators in Java using custom annotations and reflection. 

- We define a custom annotation `@Decorator` that can be used to mark methods as decorators.
- We create a class `ExampleClass` with a method `exampleMethod` that we want to decorate.
- We define a decorator class `ExampleDecorator` with a static method `decoratorMethod` that wraps the execution of a method.
- In the `Main` class, we use reflection to get the `exampleMethod` from `ExampleClass`.
- We check if the method has the `@Decorator` annotation using `isAnnotationPresent`.
- If the method is decorated, we get the `decoratorMethod` from `ExampleDecorator` using reflection.
- Finally, we invoke the `decoratorMethod` passing the `exampleMethod` as an argument.

When you run the code, it will print:
```
Before executing exampleMethod
Executing exampleMethod
After executing exampleMethod
```
This demonstrates how the decorator wraps the execution of the decorated method.

In [1]:
import java.lang.annotation.*;
import java.lang.reflect.Method;

// Define a custom annotation for function decorators
@Retention(RetentionPolicy.RUNTIME)
@Target(ElementType.METHOD)
@interface Decorator {
}

// Define a class with a method to be decorated
class ExampleClass {
    @Decorator
    public static void exampleMethod() {
        System.out.println("Executing exampleMethod");
    }
}

// Define a decorator class that wraps the execution of a method
class ExampleDecorator {
    public static void decoratorMethod(Method method) {
        System.out.println("Before executing " + method.getName());
        try {
            method.invoke(null);
        } catch (Exception e) {
            e.printStackTrace();
        }
        System.out.println("After executing " + method.getName());
    }
}

public class Main {
    public static void main(String[] args) throws NoSuchMethodException {
        // Get the exampleMethod from ExampleClass
        Method exampleMethod = ExampleClass.class.getMethod("exampleMethod");

        // Check if the method has the Decorator annotation
        if (exampleMethod.isAnnotationPresent(Decorator.class)) {
            // Get the decorator method from ExampleDecorator
            Method decoratorMethod = ExampleDecorator.class.getMethod("decoratorMethod", Method.class);

            // Invoke the decorator method passing the exampleMethod
            try {
                decoratorMethod.invoke(null, exampleMethod);
            } catch (Exception e) {
                e.printStackTrace();
            }
        }
    }
}

Main.main(null);

Before executing exampleMethod
Executing exampleMethod
After executing exampleMethod


# Class Annotations

In Java, class decorators are implemented using annotations. Annotations provide a way to associate metadata with classes, methods, fields, etc. In this example, we define a custom annotation `@MyAnnotation` with a single value `value()`. The `@Retention` annotation specifies that the annotation should be retained at runtime, and the `@Target` annotation specifies that the annotation can be applied to classes.

We then use the `@MyAnnotation` decorator to annotate the `MyClass` class. The annotation value is set to "ClassDecorator". 

In the `Main` class, we demonstrate how to access the annotation value using reflection. We retrieve the `MyAnnotation` annotation from the `MyClass` class and print its value.

Finally, we create an instance of `MyClass` and call the `printName` method to demonstrate the usage of the class decorator.

Expected output:
```
Annotation value: ClassDecorator
Name: John
```

In [2]:
import java.lang.annotation.*;

// Define a custom annotation
@Retention(RetentionPolicy.RUNTIME)
@Target(ElementType.TYPE)
@interface MyAnnotation {
    String value();
}

// Define a class decorator using the custom annotation
@MyAnnotation("ClassDecorator")
class MyClass {
    private String name;

    public MyClass(String name) {
        this.name = name;
    }

    public void printName() {
        System.out.println("Name: " + name);
    }
}

public class Main {
    public static void main(String[] args) {
        // Get the annotation value using reflection
        Class<MyClass> myClass = MyClass.class;
        MyAnnotation annotation = myClass.getAnnotation(MyAnnotation.class);
        System.out.println("Annotation value: " + annotation.value()); // Expected output: Annotation value: ClassDecorator

        // Create an instance of MyClass and call the printName method
        MyClass obj = new MyClass("John");
        obj.printName(); // Expected output: Name: John
    }
}

Main.main(null);

Annotation value: ClassDecorator
Name: John


# Useful Built-In Annotations

## Built-in Annotations in `java.lang`

Java provides several annotations in the `java.lang` package. Here's a brief overview of some of the most commonly used annotations from this package:

### 1. `@Override`

- **Description**: 
  Indicates that a method declaration is intended to override a method declaration in a superclass.
  
- **Effect**: 
  If the method doesn't properly override a method in one of its superclasses, the compiler generates an error.

### 2. `@Deprecated`

- **Description**: 
  Marks a program element as no longer being recommended, suggesting there might be a better alternative.
  
- **Effect**: 
  When a program uses a method, field, or class annotated with `@Deprecated`, the compiler generates a warning.

### 3. `@SuppressWarnings`

- **Description**: 
  Tells the compiler to suppress specific warnings.
  
- **Effect**: 
  It requires a parameter, which is the name of the warning to suppress. For example, `@SuppressWarnings("unchecked")` suppresses unchecked warnings.

### 4. `@SafeVarargs`

- **Description**: 
  Applied to a method or constructor that takes varargs parameters. 
  
- **Effect**: 
  Asserts that the code does not perform potentially unsafe operations on its varargs parameter. It's used when the compiler generates an unchecked warning, but the developer knows the method's implementation is safe and won't cause heap pollution.

### 5. `@FunctionalInterface`

- **Description**: 
  Indicates an interface is intended to be a functional interface (an interface with a single abstract method).
  
- **Effect**: 
  While optional, it's a best practice to use this annotation for functional interfaces. It ensures that the interface remains functional in case someone mistakenly adds another abstract method.

---

These annotations mainly instruct the compiler or aid in documentation. While they don't alter program execution, they can produce warnings during compilation or help in generating appropriate documentation.

# More Annotation Syntax

Key Points:
- you have to import a lot of things to make it work
  - all from `java.lang.annotation`
    - enum value `java.lang.annotation.ElementType` for targets
- `@Target` below is taking an __array__ of enum values
  - Java allows you to ommit the `new ElementType[]` part for annotations
- `@Target` and `@Retention` are __meta-annotations__ (applied to annotations)

In [3]:
// The necessary imports to make the code work
import java.lang.annotation.Retention;
import java.lang.annotation.RetentionPolicy;
import static java.lang.annotation.ElementType.FIELD;
import static java.lang.annotation.ElementType.METHOD;
import static java.lang.annotation.ElementType.PARAMETER;
import java.lang.annotation.Target;

// Defining the annotation
@Target({ FIELD, PARAMETER, METHOD }) // Array of enum values
@Retention(RetentionPolicy.RUNTIME) // Keep at runtime for libraries
public @interface MyAnnotation {} // The actual name of the annotation

class MyClass {
  @MyAnnotation int n; // I can use this here because of my @Retention
  // Anyone doing runtime reflection on MyClass will see that the variable
  // n has been annotated with this annotation.
}

# Annotations That Take Parameters

Key Points:
- parameters are __methods of the interface__
- `default` is a special keyword for annotation methods that have default values
- never need `new Type[]` syntax for annotation arrays like in normal code
- annotations use __keyword arguments__ unlike methods

In [4]:
// Defining the annotation
public @interface DemoAnnotation {

    // #1: Default Values
    int number() default 5;
    
    String text() default "defaultText";

    // Array attribute with default value
    int[] numberArray() default {2, 3};

    // Array attribute without default value
    String[] textArray();

    // #3: Array Syntax Simplification (shown in usage below)
    // When providing more than one value, you can omit the new TypeName[] syntax.

    // #4: Single-Value Annotations
    // If this was the only element and named `value`, we could omit the name when using it.
    String value();
}

// Using the annotation

// Providing only the mandatory 'value' attribute. 
// Others will take default values where provided.
@DemoAnnotation(value = "singleValue", textArray = {"arrayItem1", "arrayItem2"})
public class AnnotatedClass {
    //...
}

# Annotation Restrictions

- must take __constant value__ (which can include enum values)
- __no inheritance__ and __no generics__
- __no custom types__ (only certain types like String, numbers, and enum values allowed)
- __no exceptions__
- __no circular references__

# Single Parameter Annotation

An annotation with a single field called `value` can ommit the name in the use of the annotation.

In [5]:
public @interface SingleValueAnnotation {
    String value();  // This is named 'value'
}

public @interface MultiValueAnnotation {
    String name();
    int age();
}

// Using the annotations

@SingleValueAnnotation("This is the value") // No need to specify the element name
public class AnnotatedClass1 { }

@MultiValueAnnotation(name = "John", age = 30) // Element names must be specified
public class AnnotatedClass2 { }

# Annotating Method Parameters

In [8]:
public @interface Logged {
    String value() default "Logging this parameter";
}

public @interface MaxValue {
    int value();
}

public class AnnotatedMethods {

    public void greet(@Logged String message) {
        System.out.println(message);
    }

    public void setLimit(@MaxValue(100) int limit) {
        System.out.println("Limit set to: " + limit);
    }
}

public class Main {
    public static void main(String[] args) {
        AnnotatedMethods am = new AnnotatedMethods();
        am.greet("Hello, World!");  // If a framework reads the @Logged annotation, it might log this parameter value.
        am.setLimit(50);            // If a framework reads the @MaxValue annotation, it might validate that the value is <= 100.
    }
}

# Default Retention & Target

If you don't provide these meta-annotations, the defaults are:
- `RetentionPolicy.CLASS` which means in the .class file but not at runtime
- __all targets__ allowed